In [ ]:
from fund_lens_etl.config import USState
from fund_lens_etl.flows.bronze_ingestion_flow import bronze_ingestion_flow
from datetime import date

In [ ]:
# Test with a single committee first
result = bronze_ingestion_flow(
    state=USState.MD,
    election_cycle=2026,
    committee_ids=["C00840017"],  # One of the committees that worked in our test
)

print("\n" + "="*70)
print("RESULTS:")
print("="*70)
for key, value in result.items():
    if key != "committee_details":
        print(f"{key}: {value}")

In [ ]:
from fund_lens_etl.database import get_session
from sqlalchemy import text

with get_session() as session:
    # Check committees
    committees = session.execute(
        text("SELECT COUNT(*) FROM bronze_fec_committee")
    ).scalar()

    # Check candidates
    candidates = session.execute(
        text("SELECT COUNT(*) FROM bronze_fec_candidate")
    ).scalar()

    # Check contributions
    contributions = session.execute(
        text("SELECT COUNT(*) FROM bronze_fec_schedule_a")
    ).scalar()

    print(f"Database counts:")
    print(f"  Committees: {committees}")
    print(f"  Candidates: {candidates}")
    print(f"  Contributions: {contributions}")

In [ ]:
from fund_lens_etl.database import get_session
from sqlalchemy import text

with get_session() as session:
    # Get sample contributions
    result = session.execute(
        text("""
            SELECT
                sub_id,
                contributor_name,
                contribution_receipt_amount,
                contribution_receipt_date,
                committee_id
            FROM bronze_fec_schedule_a
            LIMIT 5
        """)
    )

    print("Sample contributions:")
    for row in result:
        print(f"  {row.contributor_name}: ${row.contribution_receipt_amount} on {row.contribution_receipt_date}")

In [ ]:
# Full run - all MD candidate committees
result = bronze_ingestion_flow(
    state=USState.MD,
    election_cycle=2026,
    # No committee_ids = process all candidate committees
)

print("\n" + "="*70)
print("FULL RUN RESULTS:")
print("="*70)
for key, value in result.items():
    if key != "committee_details":
        print(f"{key}: {value}")

# Show per-committee breakdown
print("\nPer-Committee Breakdown:")
for detail in result["committee_details"]:
    print(f"  {detail['committee_name']}: {detail['total_records']:,} contributions")